## TensorFlow Linear

In [35]:
import tensorflow as tf

csvFile = 'pima-indians-diabetes.csv'

In [39]:
learning_rate = 0.00005
epoch = 1
batch_size = 32
filename_queue = tf.train.string_input_producer([csvFile])
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)
record_defaults = [[1.], [1.], [1.], [1.], [1.], [1.], [1.], [1.], [1]]

features = tf.decode_csv(value, record_defaults=record_defaults)
X_data = features[:-1]
Y_data = features[-1:]
# batch_xs, label = tf.train.shuffle_batch(tensors=[X_data, Y_data], batch_size=batch_size,
#                                          num_threads=1,
#                                          capacity=5000,
#                                          min_after_dequeue=100)
batch_xs, label = tf.train.batch(tensors=[X_data, Y_data], batch_size=batch_size)
label = tf.one_hot(label, depth=2) # 0 => [1,0] 1 => [0,1]
label = tf.reshape(label,[-1,2])
features = len(X_data)
classes = 2
print(type(features), type(classes))

<class 'int'> <class 'int'>


In [42]:
X = tf.placeholder(tf.float32, [None, features], name="input")
Y = tf.placeholder(tf.float32, [None, classes], name="output")

W = tf.Variable(tf.random_normal([features, classes]))
b = tf.Variable(tf.random_normal([classes], name='bias'))
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.arg_max(hypothesis,1), tf.arg_max(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [43]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(tf.global_variables_initializer())
    
    for i in range(epoch):
        for step in range(2001):
            opt, cc = sess.run([optimizer,cost], feed_dict={X:batch_xs.eval(), Y:label.eval()})
    #         print(batch_xs.eval(),label.eval())
            if step%20 ==0:
                atemp = accuracy.eval({X:batch_xs.eval(), Y:label.eval()})
                print(step, cc, atemp)
    
#     opt, cc,hy = sess.run([optimizer,cost, hypothesis], feed_dict={X:batch_xs.eval(), Y:label.eval()})
#     print(batch_xs.eval(),W.eval(),hy)
    coord.request_stop()
    coord.join(threads)

0 nan 0.65625
20 nan 0.625
40 nan 0.75
60 nan 0.53125
80 nan 0.65625
100 nan 0.625
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: shuffle_batch_4/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_4/random_shuffle_queue, shuffle_batch_4/packed, shuffle_batch_4/packed_1)]]

Caused by op 'shuffle_batch_4/random_shuffle_queue_enqueue', defined at:
  File "D:\python3_5anaconda\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\python3_5anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\python3_5anaconda\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\python3_5anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.st

KeyboardInterrupt: 